# Setup Unity Catalog: Hands-on Guide

**Objective:** Enable Unity Catalog for our Databricks workspace. This involves creating a **Metastore**, setting up cloud storage for that metastore, and assigning the metastore to our workspace.

---

## 1. Prerequisites Checklist
Before we begin, ensure you have the following ready in your Azure Portal:
1.  **Resource Group:** The one we created earlier (e.g., `self-adb-rg`).
2.  **Databricks Workspace:** Created and running.
3.  **Permissions:** You must be an **Account Admin** in Databricks and have **Owner/Contributor** rights on the Azure Subscription/Resource Group.

---

## 2. Step-by-Step Setup Guide

### Step A: Create Storage Account for Metastore (Azure Portal)
Unity Catalog needs a place to store the actual data for "Managed Tables".
1.  Go to **Azure Portal** -> **Create a resource** -> **Storage account**.
2.  **Resource Group:** `self-adb-rg`.
3.  **Name:** Unique name (e.g., `adbself01`).
4.  **Region:** **MUST MATCH** your Databricks Workspace region (e.g., `Central India`).
5.  **Advanced Tab:** Enable **Hierarchical namespace** (ADLS Gen2). **This is mandatory.**
6.  Review & Create.

### Step B: Create Container
1.  Go to the new Storage Account -> **Containers**.
2.  Create a container named `root` (or `metastore`).
3.  Inside `root`, create a directory named `metastore`.

### Step C: Create Access Connector for Azure Databricks
This is a specific Azure resource that allows Unity Catalog to access the storage account securely.
1.  Search for **Access Connector for Azure Databricks**.
2.  Click **Create**.
3.  **Resource Group:** `self-adb-rg`.
4.  **Name:** `adb-uc-connector`.
5.  **Region:** Same as workspace (`Central India`).
6.  Review & Create.
7.  **Important:** Once created, go to the resource and copy the **Resource ID**. It looks like `/subscriptions/.../providers/Microsoft.Databricks/accessConnectors/...`.

### Step D: Grant Permissions (IAM)
The Access Connector needs permission to talk to the Storage Account.
1.  Go to your **Storage Account** (`adbself01`).
2.  Click **Access Control (IAM)** -> **Add role assignment**.
3.  **Role:** `Storage Blob Data Contributor`.
4.  **Assign access to:** `Managed Identity`.
5.  **Select Members:** Choose the **Access Connector** created in Step C.
6.  Review & Assign.

---

## 3. Configure Unity Catalog (Databricks Account Console)

Now we switch to the **Databricks Account Console** (`https://accounts.azuredatabricks.net`).

1.  Go to **Catalog** tab.
2.  Click **Create Metastore**.
3.  **Name:** `azure-central-india` (Naming convention: cloud-region).
4.  **Region:** `Central India`.
5.  **ADLS Gen 2 Path:**
    *   Format: `abfss://<container>@<storage-account>.dfs.core.windows.net/<path>`
    *   Example: `abfss://root@adbself01.dfs.core.windows.net/metastore`
6.  **Access Connector ID:** Paste the **Resource ID** copied in Step C.
7.  Click **Create**.

### Step E: Assign to Workspace
1.  After creating the metastore, you will be prompted to assign workspaces.
2.  Select your workspace (`self-adb`).
3.  Click **Assign**.
4.  Click **Enable**.

---

## 4. Verification

1.  Go back to your **Databricks Workspace**.
2.  Click on **Catalog** in the left sidebar.
3.  You should now see a catalog named `main` (or the default catalog created by UC) alongside `hive_metastore`.
4.  **Success!** Unity Catalog is now active.

In [ ]:
# Verification Code
# Run this in your Databricks Workspace Notebook to confirm UC is enabled.

try:
    # List available catalogs. If UC is enabled, you will see more than just 'hive_metastore'
    df = spark.sql("SHOW CATALOGS")
    display(df)
    
    print("If you see 'main' or 'system' catalogs, Unity Catalog is ENABLED.")
except Exception as e:
    print("Error checking catalogs. Ensure cluster is running.")
    print(e)

## 5. Key Takeaways
*   **One Metastore per Region:** You generally create one metastore for a specific region and attach all workspaces in that region to it.
*   **Managed Identity:** We used a secure Access Connector (Managed Identity) instead of access keys. This is the secure, modern way to connect Azure resources.
*   **Root Storage:** The ADLS path we configured is the default home for all "Managed Tables" created in this metastore.

---

## Next Steps
Now that Unity Catalog is set up, in the next session, we will start creating **Catalogs, Schemas, and Tables** within Unity Catalog and learn about managing permissions.